In this case we shall be using neural networks as meta-learner for three sub-models. Integrated Stacking is basically a way of aggregating all the prdictions of the preditors in an ensemble using deep neural nets. The output predictions of all the submodels are used as input for the meta-learner. By doing this the accuracy of the final models increase.



In [20]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle

In [2]:
a = []

In [3]:
X_test = pd.read_csv('dataset/X_test.csv')
y_test = pd.read_csv('dataset/y_test.csv')

In [4]:
X_test.head()

,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_count,credits,keywords
0,Barbara Hammer Lends a Hand,NaN,en,NaN,0.600,NaN,28-03-2012,0,0,1.0,Released,NaN,0,Barbara Hammer,NaN
1,Fuck Private Property!,Drama,en,France 2045. In the midst of a food crisis Ale...,0.600,NaN,01-02-2022,0,0,15.0,Released,NaN,0,NaN,NaN
2,FEMMEfille,Documentary,fr,The story of Isabelle Caro Oliviero Toscani's ...,0.600,Tag/Traum Filmproduktion,29-10-2014,0,0,0.0,Released,NaN,1,Kiki Allgeier-Oliviero Toscani-Isabelle Caro,anorexia
3,My Mother Is a Parrot,NaN,es,Juana an eleven-year-old girl comes from a ver...,1.400,Masa Latina,12-10-2017,0,0,81.0,Released,NaN,2,Valentina Marcone-Gabriel Páez-Natalia Señoral...,NaN
4,A Prairie Home Companion,Drama-Comedy-Music,en,A look at what goes on backstage during the la...,7.849,Sandcastle 5-Prairie Home Productions-Ivanhoe ...,09-06-2006,10000000,0,105.0,Released,Radio like you've never seen it before.,241,Meryl Streep-Lily Tomlin-Lindsay Lohan-Garriso...,country music-commercial-minnesota-radio prese...


In [ ]:
X_test.dropna(inplace=True)

In [5]:
def data_Converter(data):
    X1 = data[['vote_count', 'runtime', 'revenue', 'budget', 'popularity', 'original_language', 'status']]
    X2 = data[['genres','credits']]
    X3 = data[['overview']]
    X4 = data[['keywords']]
    return X1, X2, X3, X4

In [6]:
X_model = X_test.iloc[:-1]
X_predict = X_test.iloc[X_test.shape[0]-1]
y_model = y_test.iloc[:-1]
y_predict = y_test.iloc[y_test.shape[0]-1]

In [7]:
X_model.shape
y_model.shape

(1,)

In [ ]:
category_tranformer = pickle.load(open('dataset/category_tranformer.pkl', 'rb'))

In [ ]:
category_tranformer_2 = pickle.load(open('dataset/category_tranformer_2.pkl', 'rb'))


In [23]:
tokenizer = pickle.load(open('/home/dwip.dalal/movies_rating_prediction/tokenizer.pickle', 'rb'))
tokenizer_2 = pickle.load(open('/home/dwip.dalal/movies_rating_prediction/tokenizer_2.pickle', 'rb'))
tokenizer_3 = pickle.load(open('/home/dwip.dalal/movies_rating_prediction/tokenizer_3.pickle', 'rb'))
tokenizer_4 = pickle.load(open('/home/dwip.dalal/movies_rating_prediction/tokenizer_4.pickle', 'rb'))

In [ ]:
X1, X2, X3, X4 = data_Converter(X_predict)

In [ ]:
maxi = 0
def find_len(sequence_train):
    for i in sequence_train:
        maxi = max(len(i), maxi)

In [ ]:
X1['original_language'] = category_tranformer.transform(X1['original_language'])
X1['status'] = category_tranformer_2.transform(X1['status'])
sequence_train = tokenizer.texts_to_sequences(X2['genres'])
sequence_train_2 = tokenizer_2.texts_to_sequences(X2['credits'])
sequence_train_3 = tokenizer_3.texts_to_sequences(X3['overview'])
sequence_train_4 = tokenizer_4.texts_to_sequences(X4['keywords'])

X2['genres'] = pad_sequences(sequence_train, maxlen= find_len(sequence_train))
X2['credits'] = pad_sequences(sequence_train_2, maxlen= find_len(sequence_train_2))
X3['overview'] = pad_sequences(sequence_train_3, maxlen= find_len(sequence_train_3))
X4['keywords'] = pad_sequences(sequence_train_4, maxlen= find_len(sequence_train_4))

In [8]:
model1 = tf.keras.models.load_model('/home/dwip.dalal/movies_rating_prediction/Weights-048--0.61986.hX_train5')
model2 = tf.keras.models.load_model('/home/dwip.dalal/movies_rating_prediction/Weights_2_-071--0.71807.hX_train5')
model3 = tf.keras.models.load_model('/home/dwip.dalal/movies_rating_prediction/Weights_3_-062--0.65039.hX_train5')
model4 = tf.keras.models.load_model('/home/dwip.dalal/movies_rating_prediction/Weights_4_-112--0.55039.hX_train5')

In [9]:
model1_layer_names = ['dense_2', 'dense_3', 'dense_4', 'dense_5']
model2_layer_names = ['embedding', 'lstm', 'lstm_1', 'dense_6', 'dense_7', 'dense_8']
model3_layer_names = ['dense_9', 'dense_10', 'dense_11']
model4_layer_names = ['embedding_2', 'lstm_2', 'lstm_1_2', 'dense_12', 'dense_13', 'dense_14']

In [10]:
p = 0
for layer in model1.layers:
  layer._name = model1_layer_names[p]
  p += 1
p = 0
for layer in model2.layers:
  layer._name = model2_layer_names[p]
  p += 1
p = 0
for layer in model3.layers:
  layer._name = model3_layer_names[p]
  p += 1
p = 0
for layer in model4.layers:
  layer._name = model4_layer_names[p]
  p += 1

In [11]:
model1.layers[0].input._name = 'dense_input_1'
model4.layers[0].input._name = 'embedding_input_1'

In [12]:
model1.layers[0].input._name 

'dense_input_1'

In [13]:
model1._input_layers[0]._name = "dense_input_1"
model4._input_layers[0]._name = 'embedding_input_1'


In [14]:
a.append(model1)
a.append(model2)
a.append(model3)
a.append(model4)

In [15]:
def stacked_model(members):
	# here we don't want to upload the weights of our already pretrained model so we have to upadte all layers in all models to not be trainable.
  # print(members)
  for i in range(len(members)):
    model = members[i]
    for layer in model.layers:
      layer.trainable = False
			
	# define multi-headed input
  ensemble_visible = [model.input for model in members]
  print(ensemble_visible)
  # concatenate merge output from each model
  ensemble_outputs = [model.output for model in members]
  print(ensemble_outputs)
  merge = Concatenate()(ensemble_outputs)

  hidden = keras.layers.Dense(7, activation='relu')(merge)
  output = keras.layers.Dense(1, activation='linear')(hidden)
  model = tf.keras.Model(inputs=ensemble_visible, outputs=output)
  plot_model(model, show_shapes=True, to_file='model.png')
  
  model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
  return model

In [16]:
final_model = stacked_model(a)

[<KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'dense_input_1')>, <KerasTensor: shape=(None, 52) dtype=float32 (created by layer 'embedding_input')>, <KerasTensor: shape=(None, 653) dtype=float32 (created by layer 'dense_input')>, <KerasTensor: shape=(None, 52) dtype=float32 (created by layer 'embedding_input_1')>]
[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_5')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_8')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_11')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_14')>]


In [17]:
final_model.input

[<KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'dense_input_1')>,
 <KerasTensor: shape=(None, 52) dtype=float32 (created by layer 'embedding_input')>,
 <KerasTensor: shape=(None, 653) dtype=float32 (created by layer 'dense_input')>,
 <KerasTensor: shape=(None, 52) dtype=float32 (created by layer 'embedding_input_1')>]

In [ ]:
history = final_model.fit([X1, X2, X3, X4], y_predict,epochs = 14, verbose = 2,validation_split=0.1)

### Let's make a class for making predictions

In [28]:
# data input would be:
X_predict
y_predict

vote_average    4.8
Name: 15360, dtype: float64

In [25]:
class prediction():
    def __init__(self, X_predict):
        self.X_predict = X_predict

    def split_data(self):
        self.X1, self.X2, self.X3, self.X4 = data_Converter(self.X_predict)

    def preprocess(self):
        self.X1['original_language'] = category_tranformer.transform(self.X1['original_language'])
        self.X1['status'] = category_tranformer_2.transform(self.X1['status'])
        sequence_train = tokenizer.texts_to_sequences(self.X2['genres'])
        sequence_train_2 = tokenizer_2.texts_to_sequences(self.X2['credits'])
        sequence_train_3 = tokenizer_3.texts_to_sequences(self.X3['overview'])
        sequence_train_4 = tokenizer_4.texts_to_sequences(self.X4['keywords'])

        self.X2['genres'] = pad_sequences(sequence_train, maxlen= find_len(sequence_train))
        self.X2['credits'] = pad_sequences(sequence_train_2, maxlen= find_len(sequence_train_2))
        self.X3['overview'] = pad_sequences(sequence_train_3, maxlen= find_len(sequence_train_3))
        self.X4['keywords'] = pad_sequences(sequence_train_4, maxlen= find_len(sequence_train_4))
        
    def predict(self):
        return final_model.predict([self.X1, self.X2, self.X3, self.X4])


In [ ]:
prediction.predict(X_predict)